In [1]:
import os
import json
import json
import regex as re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import pandas as pd
import string
from pyvi.ViTokenizer import tokenize

In [20]:
_WORD_SPLIT = re.compile("([.,!?\"/':;)(])")
_DIGIT_RE = re.compile(br"\d")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")

def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ' and w not in string.punctuation]

def remove_appending_title(text,title):
    return text.replace(f"{title}\n\n{title}",f"{title} ")

def create_sliding_window(text, size=256, overlap=32):
    actual_size = size - overlap
    windows = []
    n_windows = math.ceil(len(text)/actual_size)
    for i in range(n_windows):
        windows.append(" ".join(text[i*actual_size:i*actual_size + size]))
    return windows

In [31]:
all_titles = []
all_texts = []
all_bm25_texts = []
with open("../za-data/wikipedia_20220620_cleaned.jsonl") as f:
    for i,line in tqdm(enumerate(f)):
        text = remove_appending_title(x["text"],x["title"])
        # print(y, "|", text[:50],"\n")
        text = text.split(" ")
        sliding_windows = create_sliding_window(text)
        # bm25_windows = [" ".join(basic_tokenizer(w)) for w in sliding_windows]
        all_texts.extend(sliding_windows)
        # all_bm25_texts.extend(bm25_windows)
        all_titles.extend([x['title'],]*len(sliding_windows))

0it [00:00, ?it/s]

In [32]:
df = pd.DataFrame()
df["title"] = all_titles
df["text"] = all_texts

In [34]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=12)
df["bm25_text"] = df["text"].parallel_apply(lambda x: " ".join(basic_tokenizer(x)))

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [40]:
df.to_csv("../processed/wikipedia_20220620_cleaned_v2.csv",index=False)